## **entrega 1: reto de dentix**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from datetime import date

In [ ]:
df = pd.read_excel('Base_Dentix_anonimizada_new.xlsx')

#### **cambios en tiempo real**



1.   borré la columna de dirección porq pa eso estrá el estrato
2.   borré financiera porque tenía solo un registro
3.   borré seguro y aval porq no importan
4.   borré codigo de confirmación crédito y pagare_id porq son puros números y id's que no importan
5.   borré número credito porq es id


---

cambios después de importar


1.   borrar operaciones moneda extranjera porq nmms
2.   se borraron las de más de 40% nulos (['NO PERSONAS A CARGO', 'OTROS INGRESOS', 'CUOTA DE CREDITOS', 'TOTAL EGRESOS', 'PASIVOS', 'OPERACION MONEDA EXTRAGERA', 'DESISTIMIENTO'])
3.  borre lugar de nacimiento porq con la de  incidencia formalización tenemos
4. cambio de fecha de nacimiento a edad
5. cambiar las fechas de expedicion y aprobacion a una sola variable que diga cuanto se tardó en aprobar el crédito
6. comercial es irrelevante, se borró
7. timepo de actividad en años tiene outliers de 1900 años lol
8. no. de credito es un index y se borró






#### **revisión**

In [ ]:
df.head()

,TRANSACTION ID,FECHA NACIMIENTO,FECHA EXP DOC,NIVEL ESTUDIOS,ESTADOCIVIL,PROFESION,GÉNERO,TIPO VIVIENDA,NO PERSONAS A CARGO,ANTIGUEDAD EN LA CIUDAD tiempo en residencia,...,SEGURO,AVAL,CODIGO CONFIRMACION CREDITO,PAGARE_ID,DESISTIMIENTO,NUMERO CREDITO TESEO,SALDO CAPITAL,SALDO VENCIDO,DIAS DE MORA,FRANJA DE MORA
0,2016918,1983-07-18,2001-07-27 05:00:00,Universitario,Soltero (a),Ingeniero,M,Familiar,0.0,3.0,...,20986,69958,768277,40322901.0,NaN,768277,0,0.0,0,01. Al dia
1,2016914,1965-04-15,1983-08-16 05:00:00,Técnico,Soltero (a),Contador,F,Propia,0.0,21.0,...,7712,25710,50199432,40322628.0,NaN,50199432,5141500,0.0,0,01. Al dia
2,2016836,2005-01-25,2023-01-31 05:00:00,Técnico,Unión libre,Educador,F,Familiar,0.0,15.0,...,1889,6295,26657430,40321155.0,NaN,26657430,1259000,0.0,0,01. Al dia
3,2016736,1995-11-15,2013-11-28 00:00:00,Técnico,Unión libre,Asesor,F,Propia,0.0,7.0,...,3822,12741,81389650,40318689.0,NaN,81389650,2492576,0.0,0,01. Al dia
4,2016576,2006-11-20,2024-11-27 00:00:00,Universitario,Soltero (a),Ingeniero,F,Propia,0.0,19.0,...,2816,9387,57688111,40310628.0,NaN,57688111,1840083,0.0,0,01. Al dia


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46328 entries, 0 to 46327
Data columns (total 58 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   TRANSACTION ID                                46328 non-null  int64         
 1   FECHA NACIMIENTO                              46328 non-null  datetime64[ns]
 2   FECHA EXP DOC                                 46328 non-null  datetime64[ns]
 3   NIVEL ESTUDIOS                                46326 non-null  object        
 4   ESTADOCIVIL                                   46326 non-null  object        
 5   PROFESION                                     46326 non-null  object        
 6   GÉNERO                                        45140 non-null  object        
 7   TIPO VIVIENDA                                 46326 non-null  object        
 8   NO PERSONAS A CARGO                           22042 non-null  floa

In [ ]:
df.describe(include='all')

,TRANSACTION ID,FECHA NACIMIENTO,FECHA EXP DOC,NIVEL ESTUDIOS,ESTADOCIVIL,PROFESION,GÉNERO,TIPO VIVIENDA,NO PERSONAS A CARGO,ANTIGUEDAD EN LA CIUDAD tiempo en residencia,...,SEGURO,AVAL,CODIGO CONFIRMACION CREDITO,PAGARE_ID,DESISTIMIENTO,NUMERO CREDITO TESEO,SALDO CAPITAL,SALDO VENCIDO,DIAS DE MORA,FRANJA DE MORA
count,4.632800e+04,46328,46328,46326,46326,46326,45140,46326,2.204200e+04,46326.000000,...,46328.000000,46328.000000,4.632800e+04,4.578900e+04,149,4.632800e+04,4.632800e+04,4.632800e+04,46328.000000,46328
unique,NaN,NaN,NaN,9,5,416,3,4,NaN,NaN,...,NaN,NaN,NaN,NaN,133,NaN,NaN,NaN,NaN,9
top,NaN,NaN,NaN,Universitario,Soltero (a),Administrador,F,Familiar,NaN,NaN,...,NaN,NaN,NaN,NaN,Desistido,NaN,NaN,NaN,NaN,01. Al dia
freq,NaN,NaN,NaN,18822,30255,6923,27544,26062,NaN,NaN,...,NaN,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,32012
mean,1.483516e+06,1984-12-14 20:49:39.122776768,2003-10-25 12:07:26.036953984,NaN,NaN,NaN,NaN,NaN,1.770742e+03,10.246620,...,5739.378734,14492.891772,4.483156e+07,2.178600e+07,NaN,4.261213e+07,1.696566e+06,9.444248e+05,17.730314,NaN
min,1.173594e+06,1945-09-17 00:00:00,1948-03-18 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.500000,...,405.000000,675.000000,1.860000e+02,0.000000e+00,NaN,1.860000e+02,0.000000e+00,0.000000e+00,0.000000,NaN
25%,1.307609e+06,1974-07-03 00:00:00,1993-08-30 15:30:00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,4.000000,...,2730.000000,5291.000000,1.150800e+07,1.521833e+07,NaN,1.097004e+07,0.000000e+00,0.000000e+00,0.000000,NaN
50%,1.387568e+06,1988-01-06 00:00:00,2006-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,8.000000,...,4787.000000,9549.000000,4.100085e+07,1.905503e+07,NaN,4.036212e+07,2.539820e+05,0.000000e+00,0.000000,NaN
75%,1.664454e+06,1997-04-29 00:00:00,2015-11-17 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,19.000000,...,7401.000000,18698.000000,7.036076e+07,2.850011e+07,NaN,6.998099e+07,2.718848e+06,1.643132e+05,8.000000,NaN
max,2.049921e+06,2009-09-11 00:00:00,2025-06-04 05:00:00,NaN,NaN,NaN,NaN,NaN,3.864465e+07,21.000000,...,37500.000000,193375.000000,9.861105e+09,4.160793e+07,NaN,9.999960e+07,2.500000e+07,4.546752e+07,1559.000000,NaN


In [ ]:
df.isnull().sum()

,0
TRANSACTION ID,0
FECHA NACIMIENTO,0
FECHA EXP DOC,0
NIVEL ESTUDIOS,2
ESTADOCIVIL,2
PROFESION,2
GÉNERO,1188
TIPO VIVIENDA,2
NO PERSONAS A CARGO,24286
ANTIGUEDAD EN LA CIUDAD tiempo en residencia,2


#### **limpieza**

quitar columnas con más del 40% de datos faltantes

In [ ]:
faltantes = df.isnull().mean() * 100

mask_40 = (faltantes <= 40)

columnas_eliminadas = faltantes[faltantes > 40].index.tolist()
print(columnas_eliminadas)

['NO PERSONAS A CARGO', 'OTROS INGRESOS', 'TOTAL INGRESOS', 'CUOTA DE CREDITOS', 'TOTAL EGRESOS', 'PASIVOS', 'FINANCIERA', 'OPERACION MONEDA EXTRAGERA', 'DESISTIMIENTO']


In [ ]:
columnas_eliminadas.remove('TOTAL INGRESOS')
print(columnas_eliminadas)

['NO PERSONAS A CARGO', 'OTROS INGRESOS', 'CUOTA DE CREDITOS', 'TOTAL EGRESOS', 'PASIVOS', 'FINANCIERA', 'OPERACION MONEDA EXTRAGERA', 'DESISTIMIENTO']


In [ ]:
df = df.drop(columnas_eliminadas, axis=1)

In [ ]:
df.isnull().sum()

,0
TRANSACTION ID,0
FECHA NACIMIENTO,0
FECHA EXP DOC,0
NIVEL ESTUDIOS,2
ESTADOCIVIL,2
PROFESION,2
GÉNERO,1188
TIPO VIVIENDA,2
ANTIGUEDAD EN LA CIUDAD tiempo en residencia,2
ESTRATO,2


#### **codificación**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46328 entries, 0 to 46327
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   TRANSACTION ID                                46328 non-null  int64         
 1   FECHA NACIMIENTO                              46328 non-null  datetime64[ns]
 2   FECHA EXP DOC                                 46328 non-null  datetime64[ns]
 3   NIVEL ESTUDIOS                                46326 non-null  object        
 4   ESTADOCIVIL                                   46326 non-null  object        
 5   PROFESION                                     46326 non-null  object        
 6   GÉNERO                                        45140 non-null  object        
 7   TIPO VIVIENDA                                 46326 non-null  object        
 8   ANTIGUEDAD EN LA CIUDAD tiempo en residencia  46326 non-null  floa

In [ ]:
categoricas = df.select_dtypes(include='object').columns
print(categoricas)

Index(['NIVEL ESTUDIOS', 'ESTADOCIVIL', 'PROFESION', 'GÉNERO', 'TIPO VIVIENDA',
       'CIUDAD', 'DEPARTAMENTO', 'DIRECCION', 'BARRIO', 'ACTIVIDAD ECONÓMICA',
       'EMPRESA', 'TIPO CONTRATO', 'OCUPACIÓN', 'TIEMPO ACTIVIDAD',
       'COMERCIAL', 'CLINICA', 'LUGAR NACIMIENTO', 'INCIDENCIAFORMALIZACION',
       'FRANJA DE MORA'],
      dtype='object')


el lugar de nacimiento es lo mismo que la incidencia

In [ ]:
df.drop(['LUGAR NACIMIENTO'], axis=1, inplace=True)

In [ ]:
df.drop(['COMERCIAL'], axis=1, inplace=True)

##### fechas

cambiar fecha de nacimiento a edad

In [ ]:
df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

hoy = pd.Timestamp(date.today())
df['EDAD'] = (
    hoy.year - df['FECHA NACIMIENTO'].dt.year
    - ((hoy.month < df['FECHA NACIMIENTO'].dt.month) |
       ((hoy.month == df['FECHA NACIMIENTO'].dt.month) & (hoy.day < df['FECHA NACIMIENTO'].dt.day))))

print(df[['FECHA NACIMIENTO', 'EDAD']].head())

  FECHA NACIMIENTO  EDAD
0       1983-07-18    42
1       1965-04-15    60
2       2005-01-25    20
3       1995-11-15    29
4       2006-11-20    18


In [ ]:
df.drop(['FECHA NACIMIENTO'], axis=1, inplace=True)

cambiar las fechas de expedicion y aprobacion a una sola variable que diga cuanto se tardó en aprobar el crédito


In [ ]:
df['DIAS EN APROBAR'] = (
    pd.to_datetime(df['FECHA APROBACIÓN'], errors='coerce')
    - pd.to_datetime(df['FECHA SOLICITUD'], errors='coerce')
).dt.days

print(df[['FECHA SOLICITUD', 'FECHA APROBACIÓN', 'DIAS EN APROBAR']].head())

      FECHA SOLICITUD    FECHA APROBACIÓN  DIAS EN APROBAR
0 2025-06-17 19:17:46 2025-06-17 19:43:39                0
1 2025-06-17 19:08:42 2025-06-17 19:21:39                0
2 2025-06-17 17:46:00 2025-06-17 18:02:49                0
3 2025-06-17 16:27:25 2025-06-17 16:51:33                0
4 2025-06-17 13:54:04 2025-06-17 14:06:27                0


In [ ]:
df['DIAS EN APROBAR'].value_counts()

,count
DIAS EN APROBAR,
0,46272
1,41
2,10
3,5


In [ ]:
df['DIAS EN APROBAR'].describe()

,DIAS EN APROBAR
count,46328.000000
mean,0.001640
std,0.052126
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,3.000000


In [ ]:
variables_datetime = df.select_dtypes(include='datetime64[ns]').columns
print(variables_datetime)

Index(['FECHA EXP DOC', 'FECHA SOLICITUD', 'FECHA APROBACIÓN',
       'FECHA DESEMBOLSO', 'FCREACION DENTICUOTAS'],
      dtype='object')


In [ ]:
df.drop(variables_datetime, axis=1, inplace=True)

##### categorizar

In [ ]:
categoricas = df.select_dtypes(include='object').columns
print(categoricas)

Index(['NIVEL ESTUDIOS', 'ESTADOCIVIL', 'PROFESION', 'GÉNERO', 'TIPO VIVIENDA',
       'CIUDAD', 'DEPARTAMENTO', 'DIRECCION', 'BARRIO', 'ACTIVIDAD ECONÓMICA',
       'EMPRESA', 'TIPO CONTRATO', 'OCUPACIÓN', 'TIEMPO ACTIVIDAD', 'CLINICA',
       'INCIDENCIAFORMALIZACION', 'FRANJA DE MORA'],
      dtype='object')


que onda con la de tiempo en años

In [ ]:
df['TIEMPO ACTIVIDAD'].apply(type).value_counts()

,count
TIEMPO ACTIVIDAD,
<class 'int'>,36246
<class 'float'>,9573
<class 'str'>,445
<class 'datetime.datetime'>,64


eran 445 datos raros que ya borramos :) más otros 64 de fechas

In [ ]:
df = df[~df['TIEMPO ACTIVIDAD'].apply(lambda x: isinstance(x, str))]

In [ ]:
import datetime

In [ ]:
df = df[~df['TIEMPO ACTIVIDAD'].apply(lambda x: isinstance(x, datetime.datetime))]

In [ ]:
df['TIEMPO ACTIVIDAD'] = df['TIEMPO ACTIVIDAD'].astype(float)

##### objects

In [ ]:
categoricas = df.select_dtypes(include='object').columns
print(categoricas)

Index(['NIVEL ESTUDIOS', 'ESTADOCIVIL', 'PROFESION', 'GÉNERO', 'TIPO VIVIENDA',
       'CIUDAD', 'DEPARTAMENTO', 'DIRECCION', 'BARRIO', 'ACTIVIDAD ECONÓMICA',
       'EMPRESA', 'TIPO CONTRATO', 'OCUPACIÓN', 'CLINICA',
       'INCIDENCIAFORMALIZACION', 'FRANJA DE MORA'],
      dtype='object')


In [ ]:
#df.drop(['NO CREDITO'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'ANTIGUEDAD EN LA CIUDAD tiempo en residencia': 'TIMEPO EN RESIDENCIA'}, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45819 entries, 0 to 46327
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TRANSACTION ID               45819 non-null  int64  
 1   NIVEL ESTUDIOS               45817 non-null  object 
 2   ESTADOCIVIL                  45817 non-null  object 
 3   PROFESION                    45817 non-null  object 
 4   GÉNERO                       44631 non-null  object 
 5   TIPO VIVIENDA                45817 non-null  object 
 6   TIMEPO EN RESIDENCIA         45817 non-null  float64
 7   ESTRATO                      45817 non-null  float64
 8   CIUDAD                       45818 non-null  object 
 9   DEPARTAMENTO                 45818 non-null  object 
 10  DIRECCION                    45818 non-null  object 
 11  BARRIO                       45817 non-null  object 
 12  ACTIVIDAD ECONÓMICA          45817 non-null  object 
 13  EMPRESA              

In [ ]:
ordinales = ['NIVEL ESTUDIOS', 'ESTADOCIVIL', 'GÉNERO', 'TIPO VIVIENDA','TIPO CONTRATO','FRANJA DE MORA']

In [ ]:
nominales = set(categoricas) - set(ordinales)
print(nominales)

{'DIRECCION', 'CIUDAD', 'INCIDENCIAFORMALIZACION', 'EMPRESA', 'CLINICA', 'PROFESION', 'OCUPACIÓN', 'ACTIVIDAD ECONÓMICA', 'BARRIO', 'DEPARTAMENTO'}


In [ ]:
df['FRANJA DE MORA'].unique()


array(['01. Al dia', '02. 1 a 30', '03. 31 a 60', '04. 61 a 90',
       '07. 151 a 180', '06. 121 a 150', '08. 181 a 360', '05. 91 a 120',
       '09. > 360'], dtype=object)

In [ ]:
df['NIVEL ESTUDIOS'].unique()

array(['Universitario', 'Técnico', 'Tecnólogo', 'Licenciatura',
       'Bachillerato', 'Doctorado / Postdoctorado', 'Primaria',
       'Maestría', 'Especialización', nan], dtype=object)

In [ ]:
df['ESTADOCIVIL'].unique()

array(['Soltero (a)', 'Unión libre', 'Casado (a)',
       'Divorciado (a) / Separado (a)', 'Viudo (a)', nan], dtype=object)

In [ ]:
df['GÉNERO'].unique()

array(['M', 'F', 'NB', nan], dtype=object)

In [ ]:
df['TIPO VIVIENDA'].unique()

array(['Familiar', 'Propia', 'Arrendada', 'Propia con crédito', nan],
      dtype=object)

In [ ]:
df['TIPO CONTRATO'].unique()

array(['Indefinido', 'No Aplica', 'Prestación de servicios', 'Fijo', nan],
      dtype=object)

In [ ]:
mapa_mora = {
    '01. Al dia': 0,
    '02. 1 a 30': 1,
    '03. 31 a 60': 2,
    '04. 61 a 90': 3,
    '05. 91 a 120': 4,
    '06. 121 a 150': 5,
    '07. 151 a 180': 6,
    '08. 181 a 360': 7,
    '09. > 360': 8
}

df['FRANJA DE MORA'] = df['FRANJA DE MORA'].map(mapa_mora)

In [ ]:
mapa_estudios = {
    'Primaria': 1,
    'Bachillerato': 2,
    'Técnico': 3,
    'Tecnólogo': 4,
    'Universitario': 5,
    'Licenciatura': 6,
    'Especialización': 7,
    'Maestría': 8,
    'Doctorado / Postdoctorado': 9
}

df['NIVEL ESTUDIOS'] = df['NIVEL ESTUDIOS'].map(mapa_estudios)

In [ ]:
mapa_estadocivil = {
    'Soltero (a)': 0,
    'Unión libre': 1,
    'Casado (a)': 2,
    'Divorciado (a) / Separado (a)': 3,
    'Viudo (a)': 4
    }

df['ESTADOCIVIL'] = df['ESTADOCIVIL'].map(mapa_estadocivil)

In [ ]:
mapa_genero = {
    'M':0,
    'F':1,
    'NB':2
    }

df['GÉNERO'] = df['GÉNERO'].map(mapa_genero)

In [ ]:
mapa_tipovivienda = {
    'Familiar':1,
    'Propia':2,
    'Arrendada':3,
    'Propia con crédito':4
}

df['TIPO VIVIENDA'] = df['TIPO VIVIENDA'].map(mapa_tipovivienda)

In [ ]:
mapa_tipocontrato = {
    'Indefinido':0,
    'No Aplica':1,
    'Prestación de servicios':2,
    'Fijo':3
}

df['TIPO CONTRATO'] = df['TIPO CONTRATO'].map(mapa_tipocontrato)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45819 entries, 0 to 46327
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TRANSACTION ID               45819 non-null  int64  
 1   NIVEL ESTUDIOS               45817 non-null  float64
 2   ESTADOCIVIL                  45817 non-null  float64
 3   PROFESION                    45817 non-null  object 
 4   GÉNERO                       44631 non-null  float64
 5   TIPO VIVIENDA                45817 non-null  float64
 6   TIMEPO EN RESIDENCIA         45817 non-null  float64
 7   ESTRATO                      45817 non-null  float64
 8   CIUDAD                       45818 non-null  object 
 9   DEPARTAMENTO                 45818 non-null  object 
 10  DIRECCION                    45818 non-null  object 
 11  BARRIO                       45817 non-null  object 
 12  ACTIVIDAD ECONÓMICA          45817 non-null  object 
 13  EMPRESA              

In [ ]:
df = df.reset_index(drop=True)

#### **nan's**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45819 entries, 0 to 45818
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TRANSACTION ID               45819 non-null  int64  
 1   NIVEL ESTUDIOS               45817 non-null  float64
 2   ESTADOCIVIL                  45817 non-null  float64
 3   PROFESION                    45817 non-null  object 
 4   GÉNERO                       44631 non-null  float64
 5   TIPO VIVIENDA                45817 non-null  float64
 6   TIMEPO EN RESIDENCIA         45817 non-null  float64
 7   ESTRATO                      45817 non-null  float64
 8   CIUDAD                       45818 non-null  object 
 9   DEPARTAMENTO                 45818 non-null  object 
 10  DIRECCION                    45818 non-null  object 
 11  BARRIO                       45817 non-null  object 
 12  ACTIVIDAD ECONÓMICA          45817 non-null  object 
 13  EMPRESA         

In [ ]:
df = df.dropna(subset=['TIMEPO EN RESIDENCIA']).reset_index(drop=True)

In [ ]:
df = df.dropna(subset=['BARRIO']).reset_index(drop=True)

In [ ]:
df.isnull().sum()

,0
TRANSACTION ID,0
NIVEL ESTUDIOS,0
ESTADOCIVIL,0
PROFESION,0
GÉNERO,1188
TIPO VIVIENDA,0
TIMEPO EN RESIDENCIA,0
ESTRATO,0
CIUDAD,0
DEPARTAMENTO,0


In [ ]:
df_sin_nan = df.dropna()

In [ ]:
df_sin_nan.isnull().sum()

,0
TRANSACTION ID,0
NIVEL ESTUDIOS,0
ESTADOCIVIL,0
PROFESION,0
GÉNERO,0
TIPO VIVIENDA,0
TIMEPO EN RESIDENCIA,0
ESTRATO,0
CIUDAD,0
DEPARTAMENTO,0


#### **dividir df's**

In [ ]:
df_sin_nan = df_sin_nan

In [ ]:
df_numericas = df.select_dtypes(include=['int64', 'float64'])

In [ ]:
df_categoricas = df

#### **más**

ciudad con días de mora

In [ ]:
ciudad_mora = df[['CIUDAD', 'FRANJA DE MORA']]

In [ ]:
ciudad_mora.head()

,CIUDAD,FRANJA DE MORA
0,Bucaramanga,0
1,Soledad,0
2,"Bogotá, d.c.",0
3,Cali,0
4,Barranquilla,0


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

In [ ]:
# Inicializar el geocodificador
geolocator = Nominatim(user_agent="geo_app")

# Función para obtener coordenadas
def obtener_coordenadas(ciudad):
    try:
        loc = geolocator.geocode(ciudad)
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
    except:
        return pd.Series([None, None])
    return pd.Series([None, None])

# Aplicar la función a cada ciudad (con un pequeño delay por cortesía al servidor)
df[['lat', 'lon']] = ciudad_mora['CIUDAD'].apply(lambda x: obtener_coordenadas(x))
time.sleep(1)  # delay opcional si haces muchas peticiones


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [ ]:
# cargar el mapa base del mundo
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# convertir a GeoDataFrame
gdf = gpd.GeoDataFrame(
    ciudad_mora,
    geometry=gpd.points_from_xy(df['lon'], df['lat']),
    crs="EPSG:4326"
)

# graficar
fig, ax = plt.subplots(figsize=(12, 6))
world.plot(ax=ax, color='lightgray', edgecolor='white')
gdf.plot(ax=ax, color='red', markersize=gdf['FRANJA DE MORA']*3, alpha=0.7)

for i, row in gdf.iterrows():
    ax.text(row.geometry.x + 2, row.geometry.y + 2, row['CIUDAD'], fontsize=9)

plt.title("mapa mundial de valores por ciudad", fontsize=14)
plt.show()


#### **exportar**

In [ ]:
df.to_excel('avance_datos.xlsx', index=False)

In [ ]:
df_numericas.to_excel('avance_datos_numericas.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('avance_datos.xlsx')

In [ ]:
from google.colab import files
files.download('avance_datos_numericas.xlsx')

### **scaterplots**

In [ ]:
# hacer un histograma de la variable 'SALDO VENCIDO'



In [ ]:
plt.hist(df['SALDO VENCIDO'])
plt.xlabel('saldo vencido')
plt.ylabel('frecuencia')
plt.title('Histograma de Saldo Vencido')
plt.show()